### Первый способ

In [3]:
%%file customConvTranspose2d.py
#Подключение модулей
import numpy as np
import torch

def customConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0\
                          , output_padding=0, bias=True, dilation=1, padding_mode='zeros'):
    #Обёртка
    def wrapper(mat):
        
        #Проверки параметров
        if output_padding < 0:
            raise Exception(f"Padiing should be 0 or positive")
        if stride < 0:
            raise Exception(f"Stride should be 0 or positive")
        if (padding_mode != 'zeros'):
            raise Exception(f"Ivalid padding_mode")
            
        #Смещение
        if bias:
            bias_value = torch.rand(out_channels)
        else:
            bias_value = torch.zeros(out_channels)
            
        
        #Фильтр с учётом размера ядра
        if type(kernel_size) == tuple:
            flter = torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
        elif type(kernel_size) == int:
            flter = torch.rand(in_channels, out_channels, kernel_size, kernel_size)
        else:
            raise Exception(f"Ivalid kernel_size type")
            
            
        #"Обход" фильтром
        res = []
        for ochnl in range(out_channels):
            feature_map = torch.zeros((mat.shape[1] - 1) * stride + dilation * (flter.shape[2] - 1) + 1\
                                            , (mat.shape[2] - 1) * stride + dilation * (flter.shape[3] - 1) + 1)
            
            for ichnl in range(in_channels):
                for i in range(0, mat.shape[1]):
                    for j in range(0, mat.shape[2]):
                        cur = mat[ichnl][i][j]
                        val = cur * flter[ichnl][ochnl]
                        zeros = torch.zeros((flter.shape[2] - 1) * dilation + 1, (flter.shape[3] - 1) * dilation + 1)
                        for k in range(0, zeros.shape[0], dilation):
                            for f in range(0, zeros.shape[1], dilation):
                                zeros[k][f] = val[k // dilation][f // dilation]
                        total = np.add((zeros), feature_map[i * stride:i * stride + dilation * (flter.shape[2] - 1) + 1\
                                                            , j * stride:j * stride + dilation * (flter.shape[3] - 1) + 1])
                        
                        feature_map[i * stride:i * stride + dilation * (flter.shape[2] - 1) + 1\
                                    , j * stride:j * stride + dilation * (flter.shape[3] - 1) + 1] = total

            res.append(np.add(feature_map, np.full((feature_map.shape), bias_value[ochnl])))
        
        for l in range(len(res)):
            if output_padding > 0:
                pad = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
                res[l] = pad(res[l])
            res[l] = res[l][padding:res[l].shape[0] - padding, padding:res[l].shape[1] - padding]
            
        return np.array(res), np.array(flter), np.array(bias_value)
    return wrapper

Writing customConvTranspose2d.py


### Второй способ

In [19]:
%%file customConvTranspose2dSecond.py
#Подключение модулей
import numpy as np
import torch

def customConvTranspose2dSecond(in_channels, out_channels, kernel_size, stride=1, padding=0\
                              , output_padding=0, bias=True, dilation=1, padding_mode='zeros'):
    #Обёртка
    def wrapper(mat):
        #Всегда
        internal_stride = 1
        pad_size = kernel_size - 1
        
        temp = []
        for matr in mat:
            zeros = np.zeros((((matr.shape[0] - 1) * (stride) + 1), ((matr.shape[1] - 1) * (stride) + 1)))
            for i in range (0, zeros.shape[0], stride):
                for j in range (0, zeros.shape[1], stride):
                    zeros[i][j] = matr[i // (stride)][j // (stride)]
            pad = np.pad(zeros, pad_width=pad_size, mode='constant')
            temp.append(pad)
        mat = torch.tensor(np.array(temp))

        #Смещение
        if bias:
            bias_value = torch.rand(out_channels)
        else:
            bias_value = torch.zeros(out_channels)

        #Подложка
        if (padding_mode == 'zeros'):
            pad = torch.nn.ZeroPad2d(padding)
            mat = pad(mat)
        elif (padding_mode == 'reflect'):
            pad = torch.nn.ReflectionPad2d(padding)
            mat = pad(mat)
        elif (padding_mode == 'replicate'):
            pad = torch.nn.ReplicationPad2d(padding)
            mat = pad(mat)
        elif (padding_mode == 'circular'):
            pad = torch.nn.CircularPad2d(padding)
            mat = pad(mat)
        else:
            raise Exception(f"Ivalid padding_mode")

        #Фильтр с учётом размера ядра
        if type(kernel_size) == tuple:
            flter = torch.rand(out_channels, in_channels, kernel_size[0], kernel_size[1])
        elif type(kernel_size) == int:
            flter = torch.rand(out_channels, in_channels, kernel_size, kernel_size)
        else:
            raise Exception(f"Ivalid kernel_size type")

        #Инверсия фльтра для транспонированной свёртки
        flter_inv = []
        for j in range(out_channels):
            flter_in = []
            for i in range(in_channels):
                flter_in.append(np.flip(np.array(flter[j][i])))
            flter_inv.append(flter_in)
        flter_inv = np.array(flter_inv)
        flter_inv = flter_inv.reshape(in_channels, out_channels, kernel_size, kernel_size)

        #"Обход" фильтром
        res = []
        for chnl in range(out_channels):
            feature_map = np.array([])
            for i in range(0, mat.shape[1] - ((flter.shape[2]- 1) * dilation + 1) + 1, internal_stride):
                for j in range(0, mat.shape[2] - ((flter.shape[3]- 1) * dilation + 1) + 1, internal_stride):
                    total = 0
                    for k in range(in_channels):
                        cur = mat[k]\
                        [i:i + (flter.shape[2] - 1) * dilation + 1 : dilation,\
                         j:j + + (flter.shape[3] - 1) * dilation + 1 : dilation]
                        
                        total += (cur * flter[chnl][k]).sum()
                    feature_map = np.append(feature_map, float(total + bias_value[chnl]))
            res.append(feature_map.reshape((mat.shape[1] - ((flter.shape[2] - 1) * dilation + 1)) // internal_stride + 1,\
                          (mat.shape[2] - ((flter.shape[3] - 1) * dilation + 1)) // internal_stride + 1))

        return np.array(res), np.array(flter_inv), np.array(bias_value)
    return wrapper

Writing customConvTranspose2dSecond.py


### Тестирование первой реализации

In [4]:
%%file test_scenario3.py
from customConvTranspose2d import customConvTranspose2d
import numpy as np
import torch

test_data_1 = torch.rand(3, 28, 28)
test_data_2 = torch.rand(8, 5, 6)
test_data_3 = torch.rand(1, 1, 1)


def test1():
    customConv = customConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0\
                                      , output_padding=0, bias=True, dilation=3, padding_mode='zeros')
    
    result, flter, bias_value = customConv(test_data_1)
    torchConv = torch.nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0\
                                      , output_padding=0, bias=True, dilation=3, padding_mode='zeros')
    
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_1).data),2))
    assert customResult == torchResult

def test2():
    customConv = customConvTranspose2d(in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0\
                                      , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    result, flter, bias_value = customConv(test_data_2)
    torchConv = torch.nn.ConvTranspose2d(in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0\
                                      , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_2).data),2))
    assert customResult == torchResult
    
def test3():
    customConv = customConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=10, padding=0\
                                        , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    result, flter, bias_value = customConv(test_data_3)
    torchConv = torch.nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=10, padding=0\
                                        , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_3).data),2))
    assert customResult == torchResult
    
test1()
test2()
test3()

Writing test_scenario3.py


In [5]:
from customConvTranspose2d import customConvTranspose2d
import numpy as np
import torch

test_example = torch.rand(5, 6, 6)

customConv = customConvTranspose2d(in_channels=5, out_channels=1, kernel_size=3, stride=3, padding=5\
                                    , output_padding=2, bias=True, dilation=1, padding_mode='zeros')

result, flter, bias_value = customConv(test_example)
torchConv = torch.nn.ConvTranspose2d(in_channels=5, out_channels=1, kernel_size=3, stride=3, padding=5\
                                    , output_padding=2, bias=True, dilation=1, padding_mode='zeros')

torchConv.weight.data = torch.tensor(flter)
torchConv.bias.data = torch.tensor(bias_value)

print(result)
print("-------------------------------------------------")
print(np.array(torchConv(test_example).data))

[[[1.7792922 2.5157301 2.110515  1.8895226 2.2441175 1.576329  1.8156123
   2.4837356 1.8313406 1.7373867]
  [2.0896811 1.3205748 1.6990821 2.3674774 1.5113062 1.6117404 1.8462002
   1.5907941 1.9376338 2.0887392]
  [2.2656255 1.4315388 2.15599   2.0114903 1.3795806 1.999511  1.926034
   1.7563603 2.1904974 2.2462597]
  [1.8790402 1.8149948 1.5570121 1.7175183 1.9306743 1.4679375 1.329545
   2.1409996 1.6776049 1.4884355]
  [3.003158  1.1435169 1.5611869 1.6575451 1.4530227 1.6300819 2.0542629
   1.954143  2.414229  2.4341228]
  [2.8341527 1.1874142 1.4745773 1.8439257 1.4826243 2.0956154 1.913441
   2.0704186 2.842034  2.6833298]
  [2.1066668 1.4244044 1.4119723 1.1940478 1.9133489 1.4835664 1.5014807
   2.8111885 1.9930539 1.8734798]
  [2.312529  1.574811  1.733932  2.1550362 1.2990253 1.6525042 2.093441
   1.6003478 2.0818734 2.2449143]
  [2.2275567 1.6147326 2.2878418 2.0181699 1.5554805 2.0487187 1.7959723
   1.6388668 2.3478105 2.379274 ]
  [1.6523278 2.1104898 1.5549833 1.583211

### Тестирование второй реализации

In [20]:
%%file test_scenario4.py
from customConvTranspose2dSecond import customConvTranspose2dSecond
import numpy as np
import torch

test_data_1 = torch.rand(1, 28, 28)
test_data_2 = torch.rand(3, 5, 6)
test_data_3 = torch.rand(1, 1, 1)


def test1():
    customConv = customConvTranspose2dSecond(in_channels=1, out_channels=1, kernel_size=1, stride=10, padding=0\
                                            , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    result, flter, bias_value = customConv(test_data_3)
    torchConv = torch.nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=10, padding=0\
                                            , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_3).data),2))
    assert customResult == torchResult

def test2():
    customConv = customConvTranspose2dSecond(in_channels=3, out_channels=1, kernel_size=2, stride=2, padding=0\
                                      , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    result, flter, bias_value = customConv(test_data_2)
    torchConv = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=2, stride=2, padding=0\
                                      , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_2).data),2))
    assert customResult == torchResult
    
def test3():
    customConv = customConvTranspose2dSecond(in_channels=1, out_channels=2, kernel_size=4, stride=3, padding=0\
                                              , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    result, flter, bias_value = customConv(test_data_1)
    torchConv = torch.nn.ConvTranspose2d(in_channels=1, out_channels=2, kernel_size=4, stride=3, padding=0\
                                              , output_padding=0, bias=True, dilation=1, padding_mode='zeros')
    
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_1).data),2))
    assert customResult == torchResult
    
test1()
test2()
test3()

Writing test_scenario4.py


In [21]:
from customConvTranspose2dSecond import customConvTranspose2dSecond
import numpy as np
import torch

test_example = torch.rand(1, 4, 4)

customConv = customConvTranspose2dSecond(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=0\
                                    , output_padding=0, bias=True, dilation=1, padding_mode='zeros')

result, flter, bias_value = customConv(test_example)
torchConv = torch.nn.ConvTranspose2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=0\
                                    , output_padding=0, bias=True, dilation=1, padding_mode='zeros')

torchConv.weight.data = torch.tensor(flter)
torchConv.bias.data = torch.tensor(bias_value)

print(result)
print("-------------------------------------------------")
print(np.array(torchConv(test_example).data))

[[[0.21519884 0.50199527 0.72782777 0.61293373 0.36083108 0.20347702]
  [0.44762525 1.20265044 1.38040383 1.05931298 0.54027528 0.19539903]
  [0.57751928 1.18444497 1.8691631  2.02945245 0.97209154 0.45985431]
  [0.40504532 1.28301407 2.04904416 1.80175007 1.58871806 0.4877955 ]
  [0.35438831 1.18740016 1.22136789 1.91310293 0.91064676 0.57502326]
  [0.25369319 0.51622509 0.72754092 1.21643892 0.73296237 0.8108034 ]]

 [[0.05164762 0.2551471  0.57271508 0.67190061 0.34131533 0.13472933]
  [0.1719813  0.73229625 1.41174665 1.52881745 0.97336441 0.27997581]
  [0.26250721 0.93708231 1.70653121 2.02711008 1.34197357 0.51411596]
  [0.22243614 0.7807343  1.79198926 2.10939309 1.60127857 0.86228795]
  [0.11569724 0.7766719  1.27883775 1.84348108 1.22705376 0.81300116]
  [0.10994772 0.38062305 0.55066034 0.81919962 0.48399724 0.38300196]]]
-------------------------------------------------
[[[0.21519884 0.50199527 0.7278277  0.61293375 0.36083108 0.20347703]
  [0.44762525 1.2026503  1.3804038  